# Overview

This notebook provides an overview of the `sushi` example SQLMesh projects, describing the common elements among the simple, moderate, and complex example versions.

It describes the source data used by the projects, the Python helper function that creates and adds to the data, and the differences among the versions.

# Source data

## Entities

The `sushi` examples are based on the operational data from a fictional sushi restaurant. The restaurant's activities consist of the following entities, grouped by type:

<br>

| Entity type  | Entity                     |
|--------------|----------------------------|
| People       | - Waiters <br> - Customers |
| Transactions | - Orders                   |
| Food         | - Items                    |

<br>

The entities interact as follows:

- Waiters take orders from customers
- Orders consist of 1 or more items
- Items can change price over time

## Tables

The project data correspond to an approach where data from source systems (e.g., order tracking software, HR software) is extracted and loaded into the `raw` schema before being transformed by SQLMesh models. 

The `raw` schema contains five tables. The table name, entities represented in the table, column names, column data types, column descriptions, and table grain (columns that uniquely identify each row) are: 

<br>

| Table name   | Entities represented               | Column name                                                          | Column data type                                | Column description                                 | Grain  |
|--------------|------------------------------------|----------------------------------------------------------------------|---------------------------------------------------|--------------------------------------------------|--------|
| `marketing`    | Customers                          | id <br> customer_id <br> status                                      | INT <br> INT <br> TEXT                            | Unique ID <br> Customer ID <br> Customer activity status                    | id     |
| `demographics` | Customers                          | id <br> customer_id <br> zip                                         | INT <br> INT <br> TEXT                            | Unique ID <br> Customer ID <br> Customer ZIP code                      | id     |
| `orders`       | Orders <br> Waiters <br> Customers | id <br> customer_id <br> waiter_id <br> start_ts <br> end_ts <br> ds | INT <br> INT <br> INT <br> INT <br> INT <br> TEXT | Unique ID <br> Customer ID <br> Waiter ID <br> Order start time <br> Order end time <br> Date | id, ds |
| `order_items`  | Orders <br> Items                  | id <br> order_id <br> item_id <br> quantity <br> ds                  | INT <br> INT <br> INT <br> INT <br> TEXT          | Unique ID <br> Order ID <br> Item ID <br> Item quantity <br> Date                     | id, ds |
| `items`        | Items                              | id <br> name <br> price <br> ds                                      | INT <br> TEXT <br> DOUBLE <br> TEXT               | Unique ID <br> Item name <br> Item price <br> Date                      | id, ds |

<br>

## Populating the tables

Each example project directory includes both a DuckDB database containing a small amount of data and a Python file containing functions to add data to the database.

The only function you need to call is the `add_raw_data` function. It can be called from Python or from the command line and adds data for specific dates. It takes three arguments: start date, end date, and whether the database should be reset to empty before adding the data.

Let's walk through an example of adding data to the "1_simple" project using the Python interface. First, we import the `os` module and change directories to the project folder:

In [1]:
import os
os.chdir("./1_simple")

Before adding new data, let's inspect the existing database with the Python DuckDB interface. First, we see what tables have been created:

In [2]:
import duckdb

con = duckdb.connect("db/sushi-example.db")

con.sql("SELECT table_schema, table_name, table_type FROM INFORMATION_SCHEMA.TABLES")

┌──────────────┬──────────────┬────────────┐
│ table_schema │  table_name  │ table_type │
│   varchar    │   varchar    │  varchar   │
├──────────────┼──────────────┼────────────┤
│ raw          │ marketing    │ BASE TABLE │
│ raw          │ demographics │ BASE TABLE │
│ raw          │ orders       │ BASE TABLE │
│ raw          │ items        │ BASE TABLE │
│ raw          │ order_items  │ BASE TABLE │
└──────────────┴──────────────┴────────────┘

Now let's inspect each table. 

First, we'll look at the `marketing` data on customers. Each of our 100 customers has a `customer_id` and a `status` of "ACTIVE" or "INACTIVE":

In [3]:
con.sql("SELECT * FROM raw.marketing")

┌───────┬─────────────┬──────────┐
│  id   │ customer_id │  status  │
│ int32 │    int32    │ varchar  │
├───────┼─────────────┼──────────┤
│     0 │           0 │ INACTIVE │
│     1 │           1 │ ACTIVE   │
│     2 │           2 │ ACTIVE   │
│     3 │           3 │ ACTIVE   │
│     4 │           4 │ ACTIVE   │
│     5 │           5 │ ACTIVE   │
│     6 │           6 │ ACTIVE   │
│     7 │           7 │ ACTIVE   │
│     8 │           8 │ ACTIVE   │
│     9 │           9 │ ACTIVE   │
│     · │           · │   ·      │
│     · │           · │   ·      │
│     · │           · │   ·      │
│    90 │          90 │ INACTIVE │
│    91 │          91 │ ACTIVE   │
│    92 │          92 │ ACTIVE   │
│    93 │          93 │ ACTIVE   │
│    94 │          94 │ INACTIVE │
│    95 │          95 │ INACTIVE │
│    96 │          96 │ ACTIVE   │
│    97 │          97 │ ACTIVE   │
│    98 │          98 │ ACTIVE   │
│    99 │          99 │ ACTIVE   │
├───────┴─────────────┴──────────┤
│ 100 rows (20 shown

The `demographics` table contains data on each customer's zip code:

In [4]:
con.sql("SELECT * FROM raw.demographics")

┌───────┬─────────────┬─────────┐
│  id   │ customer_id │   zip   │
│ int32 │    int32    │ varchar │
├───────┼─────────────┼─────────┤
│     0 │           0 │ 10645   │
│     1 │           1 │ 10343   │
│     2 │           2 │ 10865   │
│     3 │           3 │ 10194   │
│     4 │           4 │ 10248   │
│     5 │           5 │ 10016   │
│     6 │           6 │ 10749   │
│     7 │           7 │ 10277   │
│     8 │           8 │ 10119   │
│     9 │           9 │ 10722   │
│     · │           · │   ·     │
│     · │           · │   ·     │
│     · │           · │   ·     │
│    90 │          90 │ 10173   │
│    91 │          91 │ 10977   │
│    92 │          92 │ 10688   │
│    93 │          93 │ 10208   │
│    94 │          94 │ 10785   │
│    95 │          95 │ 10059   │
│    96 │          96 │ 10807   │
│    97 │          97 │ 10692   │
│    98 │          98 │ 10162   │
│    99 │          99 │ 10997   │
├───────┴─────────────┴─────────┤
│      100 rows (20 shown)      │
└─────────────

The `items` table contains each item for sale in the restaurant and its price on a specific date:

In [5]:
con.sql("SELECT * FROM raw.items")

┌───────┬───────────┬────────┬────────────┐
│  id   │   name    │ price  │     ds     │
│ int32 │  varchar  │ double │    date    │
├───────┼───────────┼────────┼────────────┤
│     0 │ Hotate    │   7.59 │ 2023-10-01 │
│     1 │ Tsubugai  │   6.33 │ 2023-10-01 │
│     2 │ Ahi       │    9.0 │ 2023-10-01 │
│     3 │ Sake Toro │   6.98 │ 2023-10-01 │
│     4 │ Ikura     │   3.98 │ 2023-10-01 │
│     5 │ Sake      │   7.12 │ 2023-10-01 │
│     6 │ Amaebi    │   7.41 │ 2023-10-01 │
│     7 │ Tako      │   5.59 │ 2023-10-01 │
│     8 │ Aoyagi    │   3.53 │ 2023-10-01 │
│     9 │ Escolar   │   6.56 │ 2023-10-01 │
│     · │    ·      │     ·  │     ·      │
│     · │    ·      │     ·  │     ·      │
│     · │    ·      │     ·  │     ·      │
│    16 │ Escolar   │   6.91 │ 2023-10-05 │
│    17 │ Ahi       │   3.48 │ 2023-10-05 │
│    18 │ Kani      │   3.09 │ 2023-10-05 │
│    19 │ Sake      │   4.46 │ 2023-10-05 │
│    20 │ Ebi       │    5.2 │ 2023-10-05 │
│    21 │ Maguro    │   7.89 │ 2

The `orders` table contains data on each order placed with the restaurant, including the customer placing the order, waiter taking the order, start and end times, and order date:

In [6]:
con.sql("SELECT * FROM raw.orders")

┌───────┬─────────────┬───────────┬────────────┬────────────┬────────────┐
│  id   │ customer_id │ waiter_id │  start_ts  │   end_ts   │     ds     │
│ int32 │    int32    │   int32   │   int32    │   int32    │    date    │
├───────┼─────────────┼───────────┼────────────┼────────────┼────────────┤
│     1 │          30 │         5 │ 1696145535 │ 1696145999 │ 2023-10-01 │
│     2 │          24 │         3 │ 1696120330 │ 1696120878 │ 2023-10-01 │
│     3 │          53 │         7 │ 1696181564 │ 1696181617 │ 2023-10-01 │
│     4 │          92 │         5 │ 1696128237 │ 1696130039 │ 2023-10-01 │
│     5 │          47 │         0 │ 1696137970 │ 1696139066 │ 2023-10-01 │
│     6 │          58 │         1 │ 1696154421 │ 1696156404 │ 2023-10-01 │
│     7 │          60 │         4 │ 1696161385 │ 1696162754 │ 2023-10-01 │
│     8 │           9 │         2 │ 1696149907 │ 1696152197 │ 2023-10-01 │
│     9 │          28 │         1 │ 1696188800 │ 1696190257 │ 2023-10-01 │
│    10 │          17 │  

Finally, the `order_items` table contains data on the individual items included in each order:

In [7]:
con.sql("SELECT * FROM raw.order_items")

┌───────┬──────────┬─────────┬──────────┬────────────┐
│  id   │ order_id │ item_id │ quantity │     ds     │
│ int32 │  int32   │  int32  │  int32   │    date    │
├───────┼──────────┼─────────┼──────────┼────────────┤
│     1 │        1 │      15 │        3 │ 2023-10-01 │
│     2 │        1 │       0 │        4 │ 2023-10-01 │
│     3 │        1 │       6 │        6 │ 2023-10-01 │
│     4 │        1 │      22 │        4 │ 2023-10-01 │
│     5 │        1 │       8 │        9 │ 2023-10-01 │
│     6 │        2 │       6 │        9 │ 2023-10-01 │
│     7 │        2 │      18 │        5 │ 2023-10-01 │
│     8 │        3 │      16 │        3 │ 2023-10-01 │
│     9 │        4 │      13 │        5 │ 2023-10-01 │
│    10 │        4 │      28 │        5 │ 2023-10-01 │
│     · │        · │       · │        · │     ·      │
│     · │        · │       · │        · │     ·      │
│     · │        · │       · │        · │     ·      │
│    66 │       23 │      14 │        2 │ 2023-10-05 │
│    67 │ 

In addition to the tables in the DuckDB database, the sushi projects read a "seed" CSV file containing each waiter's name. That file contains this data:

In [8]:
from addsushidata import print_file # import `print_file` from addsushidata.py

print_file("seeds/waiter_names.csv")

id,name
0,Toby
1,Tyson
2,Ryan
3,George
4,Chris
5,Max
6,Vincent
7,Iaroslav
8,Emma
9,Maia



The last date of data in the existing tables is October 5, 2023: 

In [9]:
con.sql("SELECT MAX(ds) as last_date FROM raw.orders")

┌────────────┐
│ last_date  │
│    date    │
├────────────┤
│ 2023-10-05 │
└────────────┘

We can add one more day of data with the `add_raw_data` function like this:

In [10]:
from addsushidata import add_raw_data # import `add_raw_data()` from addsushidata.py

add_raw_data(start="2023-10-06", end="2023-10-06")

Let's confirm that the additional day was added:

In [11]:
con.sql("SELECT MAX(ds) as last_date FROM raw.orders")

┌────────────┐
│ last_date  │
│    date    │
├────────────┤
│ 2023-10-06 │
└────────────┘

Finally, let's close our database connection:

In [12]:
con.close()

## Next steps

Now that you understand the sushi project data and how to add data to the project with `add_raw_data()`, you're ready to check out the examples.

Each example project's complexity level builds on the previous - for example, `2_moderate` contains modified versions of the models in `1_simple`, along with additional components. 

Fortunately, nothing bad will happen if the examples get broken, so feel free to modify and tinker. If the database is returning results that seem wrong, just delete the DuckDB files and start over with `add_raw_data()` - no harm done!